In [1]:
%pip install --upgrade gensim

     ---------------------------------------- 24.0/24.0 MB 1.1 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.0
    Uninstalling gensim-4.3.0:
      Successfully uninstalled gensim-4.3.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import re

In [17]:
raw_df = pd.read_csv("raw_data/meld/all_sent_emo.csv")
for i, d in enumerate(raw_df):
    text = raw_df.iloc[i]['Utterance']
    text_encode = text.encode("ascii", "ignore")
    text_decode = text_encode.decode()
    raw_df.at[i, 'Utterance'] = text_decode
raw_df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,also I was the point person on my companys tra...,Chandler,neutral,neutral,0,0,8,21,"00:16:16,059","00:16:21,731"
1,2,You mustve had your hands full.,The Interviewer,neutral,neutral,0,1,8,21,"00:16:21,940","00:16:23,442"
2,3,That I did. That I did.,Chandler,neutral,neutral,0,2,8,21,"00:16:23,442","00:16:26,389"
3,4,So lets talk a little bit about your duties.,The Interviewer,neutral,neutral,0,3,8,21,"00:16:26,820","00:16:29,572"
4,5,My duties? All right.,Chandler,surprise,positive,0,4,8,21,"00:16:34,452","00:16:40,917"
...,...,...,...,...,...,...,...,...,...,...,...
13703,2760,"Yeah, I mean, come on Ross, no one will even n...",Rachel,neutral,neutral,279,11,6,4,"00:14:35,457","00:14:40,211"
13704,2761,They’re not listening too me?,Ross,surprise,negative,279,12,6,4,"00:14:42,256","00:14:43,840"
13705,2762,Of course they’re listening to you! Everybody ...,Rachel,neutral,neutral,279,13,6,4,"00:14:44,008","00:14:48,511"
13706,2763,Monica you really think I should try this phas...,Ross,neutral,neutral,279,14,6,4,"00:14:48,138","00:14:52,390"


In [33]:
dialogues = raw_df.groupby('Episode')['Utterance'].apply(list)
print(dialogues[1])

['Hey.', 'Hey!', 'Hi.', 'All right, there\x92s a nuclear holocaust, I\x92m the last man on Earth. Would you go out with me?', 'Enhh', 'I\x92ve got canned goods.', 'Hey, you guys! Take a look at this!  Check this baby out, dug me a hole!', 'Excellent hole, Joe.', 'Oh no! No!! My hole!!', 'Ow!! Ow!!!', 'What?! What?!! What is it?!', 'Wow thanks!', 'Joe?', 'Yeah!', 'Uh, you\x92ve had a lot of sex right?', 'When? Today? Some, not a lot.', 'Well, it\x92s just the reason that I\x92m asking is because I kind of eh, uh, I was unable to\x97I mean I really wanted to, but I couldn\x92t\x85.', 'There huh\x97hmm, there-there was an incident.', 'Don\x92t worry about that man, that happens.', 'It\x92s happened to you?', 'Yeah! Once.', 'Well, what\x92d you do?', 'I did it anyway.', '\x91Sup? \x91Sup dude?', 'Take whatever you want, just please don\x92t hurt me.', 'So you\x92re playing a little', 'Yes, on a scale from 1 to 10, 10 being the dumbest a person can look, you are definitely 19.', 'Come on ma

In [18]:
from collections import defaultdict

stop_list = set('for a of the and to in'.split(' '))
texts = [[word for word in document.lower().split() if word not in stop_list]
         for document in dialogues[0]]

frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

processed_corpus = [[token for token in text if frequency[token] > 1] 
                    for text in texts]
print(processed_corpus)

[['i', 'on', 'my'], ['you', 'your'], ['that', 'i', 'did.', 'that', 'i', 'did.'], ['so', 'your', 'duties.'], ['my', 'all'], ['youll', 'be', 'so', 'youll', 'have', 'duties.'], ['i'], ['but', 'be', 'you', 'so', 'you', 'can', 'on'], [], ['can'], ['i'], ['all', 'have', 'you', 'on', 'but', 'i', 'i', 'can'], [], ['you', 'can'], ['my', 'he\x92s', 'lost', 'it.', 'he\x92s', 'lost', 'it.'], ['what?'], ['all', 'have', 'on'], ['so', 'can', 'that', 'on', 'can', 'be'], ['what?']]


In [19]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary<19 unique tokens: ['i', 'my', 'on', 'you', 'your']...>


In [20]:
print(dictionary.token2id)

{'i': 0, 'my': 1, 'on': 2, 'you': 3, 'your': 4, 'did.': 5, 'that': 6, 'duties.': 7, 'so': 8, 'all': 9, 'be': 10, 'have': 11, 'youll': 12, 'but': 13, 'can': 14, 'he\x92s': 15, 'it.': 16, 'lost': 17, 'what?': 18}


In [21]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
print(bow_corpus)

[[(0, 1), (1, 1), (2, 1)], [(3, 1), (4, 1)], [(0, 2), (5, 2), (6, 2)], [(4, 1), (7, 1), (8, 1)], [(1, 1), (9, 1)], [(7, 1), (8, 1), (10, 1), (11, 1), (12, 2)], [(0, 1)], [(2, 1), (3, 2), (8, 1), (10, 1), (13, 1), (14, 1)], [], [(14, 1)], [(0, 1)], [(0, 2), (2, 1), (3, 1), (9, 1), (11, 1), (13, 1), (14, 1)], [], [(3, 1), (14, 1)], [(1, 1), (15, 2), (16, 2), (17, 2)], [(18, 1)], [(2, 1), (9, 1), (11, 1)], [(2, 1), (6, 1), (8, 1), (10, 1), (14, 2)], [(18, 1)]]


In [ ]:
from gensim import models

tfidf = models.TfidfModel(bow_corpus)
# words = 

In [34]:
para = " ".join(dialogues[1])
print(para)

Hey. Hey! Hi. All right, theres a nuclear holocaust, Im the last man on Earth. Would you go out with me? Enhh Ive got canned goods. Hey, you guys! Take a look at this!  Check this baby out, dug me a hole! Excellent hole, Joe. Oh no! No!! My hole!! Ow!! Ow!!! What?! What?!! What is it?! Wow thanks! Joe? Yeah! Uh, youve had a lot of sex right? When? Today? Some, not a lot. Well, its just the reason that Im asking is because I kind of eh, uh, I was unable toI mean I really wanted to, but I couldnt. There huhhmm, there-there was an incident. Dont worry about that man, that happens. Its happened to you? Yeah! Once. Well, whatd you do? I did it anyway. Sup? Sup dude? Take whatever you want, just please dont hurt me. So youre playing a little Yes, on a scale from 1 to 10, 10 being the dumbest a person can look, you are definitely 19. Come on man, really how old?! Young! Youre a man-child okay?! Now go get changed because everybodys ready and please, oh please, keep my under

In [23]:
%pip install git+https://github.com/LIAAD/yake

  Cloning https://github.com/LIAAD/yake to c:\users\admin\appdata\local\temp\pip-req-build-93894z_3
  Resolved https://github.com/LIAAD/yake to commit 374fc1c1c19eb080d5b6115cbb8d4a4324392e54
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for yake: filename=yake-0.4.8-py2.py3-none-any.whl size=62865 sha256=2a8698581cd0a32cdb967481eff8d618bca2a270aa123e17115964504daecbf4
  Stored in directory: C:\Users\Admin\AppData\Local\Temp\pip-ephem-wheel-cache-i9dkhfld\wheels\10\9d\33\6a3358fd876c3d7c6c5c139d1496eb4b1618c7d0e15c375584
Successfully built yake
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-93894z_3'


In [39]:
import yake
kw_extractor = yake.KeywordExtractor(n = 3, dedupLim = 0.9,  dedupFunc = 'seqm',  top = 20)
keywords = kw_extractor.extract_keywords(para)
for kw, v in keywords:
    print("Keyphrase: ", kw, ": score", v)

Keyphrase:  Yeah : score 0.006870266867028622
Keyphrase:  Ross : score 0.010038869945559536
Keyphrase:  Hey : score 0.01120668041920957
Keyphrase:  Don : score 0.012826924493724592
Keyphrase:  n’t : score 0.018138955895118714
Keyphrase:  God : score 0.02050549875948391
Keyphrase:  gonna : score 0.021778542175718348
Keyphrase:  Rachel : score 0.02754476560215005
Keyphrase:  Mom : score 0.03951587175798503
Keyphrase:  Wanna : score 0.04195156917350217
Keyphrase:  Joey : score 0.04268628037231713
Keyphrase:  guys : score 0.04648389869647888
Keyphrase:  Phoebe : score 0.047600804433816576
Keyphrase:  didn : score 0.04805091468113361
Keyphrase:  married : score 0.04810648920553738
Keyphrase:  Monica : score 0.048874361622225554
Keyphrase:  back : score 0.04952723745593405
Keyphrase:  fine : score 0.05284091299538306
Keyphrase:  place : score 0.05562872998578152
Keyphrase:  baby : score 0.05594697342568458
